# Case Study: Testing Asynchronous Code



For this case study, we are going to look at how we can create a test where the value we are asserting for
and the response from the method under test do not match up with one being a `Future[Int]` and the other
being an `Int`.  

As always, we first start by making sure that we have the cat's library installed.

In [1]:
import $ivy.`org.typelevel::cats-core:2.0.0`

import $ivy.$                               

Having the library installed, lets now import our scala library and cats implicits that we will use in this notebook, namely the
imports needed to make `Traverse` and `Applicative` accessible within our notebook. 

In [7]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future

// Required to enable the higher kinded types in scala (support for F[_])
import scala.language.higherKinds

import cats.Id
import cats.Applicative
import cats.instances.future._ // for Applicative
import cats.instances.list._ // for Traverse
import cats.syntax.traverse._ // for traverse
import cats.syntax.functor._ // for map

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

// Required to enable the higher kinded types in scala (support for F[_])

import scala.language.higherKinds


import cats.Id

import cats.Applicative

import cats.instances.future._ // for Applicative

import cats.instances.list._ // for Traverse

import cats.syntax.traverse._ // for traverse

import cats.syntax.functor._ // for map

Now that we have our environment setup, lets begin by creating an example problem that we are going to solve.  

## Test Case

Our company, uptime international, provides services to support other companies track their uptime.  To that end
we have a very simple interface that users can use to obtain their uptime.  By providing us with a given hostname
we will, in turn, return the number of days that the service has been considered to be **up**.  

Our trait for this use case is pretty simple, we will provide a single function that takes in a hostname
and returns a `Future[Int]` as this will be an asynchronous request.  

In [3]:
trait UptimeClient {
  def getUptime(hostname: String): Future[Int]
}

defined trait UptimeClient

Wonderful, now lets create a sample service that can be used at a higher level, making it easier for clients with multiple hostnames to get their uptime results all at once.  

In [4]:
class UptimeService(client: UptimeClient) {
  def getTotalUptime(hostnames: List[String]): Future[Int] = hostnames.traverse(client.getUptime(_)).map(_.sum)
}

defined class UptimeService

So we have an trait for the client that will make the requests to our system, and have provided, in the library, a helper implementation that wraps a clients functionality when more complex requests are needed.  In the later case we have added some business logic, so it would behoove us to add some testing to verify:
1. Our functionality is correct
2. We do not break our functionality with future changes


## Creating Tests

To start, we want to have a test client that we could mock using a mocking framework (such as mockito) or we could create a stub implementation.  Because our client consists of only a single request, lets create a stub that will have the necessary information provided when it is created.  

In [5]:
class TestUptimeClient(hosts: Map[String, Int]) extends UptimeClient {
  def getUptime(hostname: String): Future[Int] = Future.successful(hosts.getOrElse(hostname, 0))
}

defined class TestUptimeClient

With our stubbed client, lets create a simple test to verify our functionality. 

In [10]:
def testTotalUptime() = {
  val hosts = Map("host1" -> 10, "host2" -> 6)
  val client = new TestUptimeClient(hosts)
  val service = new UptimeService(client)
  val actual = service.getTotalUptime(hosts.keys.toList)
  val expected = hosts.values.sum
  assert(expected == actual)
}

// run our test
testTotalUptime()

cmd10.sc:4: type mismatch;
 found   : cmd10.this.cmd9.TestUptimeClient
 required: ammonite.$sess.cmd3.wrapper.cmd2.UptimeClient
  val service = new UptimeService(client)
                                  ^Compilation Failed

: 

Hmm, our test failed.  Looking over our implementation it should not have had an issue.  However if we look more closely at what is happening we are comparing a `Future[Int]` that is returned from `service.getTotalUptime` with that of an `Int` with the value of **16** which we obtain from our `hosts.values.sum` code.  

Using cats there is a way that we can solve this by utilizing `Type Constructors`.  

## Cats Solution

The way to handle this issue is to provide two traits instead of one, the second trait will be one for testing and will return an `Int` instead of a `Future[Int]`.

We will create two new traits, one for the asynchronous form and one for the synchronous form. Both of these traits will inherit from `UptimeClient` however what should the return type be for the function getUptime in the base trait?

In [9]:
//type Id[A] = A

trait UptimeClient[F[_]] {
    def getUptime(hostname: String): F[Int]
}

trait AsynchronousUptimeClient extends UptimeClient[Future] {
    def getUptime(hostname: String): Future[Int]
}

trait SynchronousUptimeClient extends UptimeClient[Id] {
    def getUptime(hostname: String): Id[Int]
}

defined trait UptimeClient
defined trait AsynchronousUptimeClient
defined trait SynchronousUptimeClient

In [ ]:
import cats.Id

trait UptimeClient[F[_]] {
    def getUptime(hostname: String): F[Int]
}

trait AsynchronousUptimeClient extends UptimeClient[Future] {
    def getUptime(hostname: String): Future[Int]
}

trait SynchronousUptimeClient extends UptimeClient[Id] {
    def getUptime(hostname: String): Id[Int]
}

Alright, so we have our definition in place, lets create our TestUptimeClient to work with just an `Int`. 

In [10]:
class TestUptimeClient(hosts: Map[String, Int]) extends UptimeClient[Id] {
    def getUptime(hostname: String): Int = hosts.getOrElse(hostname, 0)
}

defined class TestUptimeClient

In [ ]:
class TestUptimeClient(hosts: Map[String, Int]) extends UptimeClient[Id] {
    def getUptime(hostname: String): Int = hosts.getOrElse(hostname, 0)
}

Now that we have the client trait and test client both updated, lets focus on updating the service to be abstracted over both of the clients.  

In [11]:
class UptimeService[F[_]](client: UptimeClient[F])(implicit a: Applicative[F]) {
    def getTotalUptime(hostnames: List[String]): F[Int] = hostnames.traverse(client.getUptime).map(_.sum)
}

defined class UptimeService

In [ ]:
class UptimeService[F[_]](client: UptimeClient[F]) {
    def getTotalUptime(hostnames: List[String]): F[Int] = ???
}

Looking at the above code, the reason that we are having a failure is that `Traverse` is expecting the type supplied to be an `Applicative` and the `Future[Int]` has an applicative, but `F[Int]` does not.  So as described in the book we need to:
> We need to prove to the compiler that F has an Applicative. 
> Do this by adding an implicit constructor parameter to UptimeService .

In [ ]:
// class UptimeService[F[_]]...

In [ ]:
class UptimeService[F[_]](client: UptimeClient[F])(implicit a: Applicative[F]) {
    def getTotalUptime(hostnames: List[String]): F[Int] = hostnames.traverse(client.getUptime).map(_.sum)
}

## Re-run Test

Alright, all of our hooks should now be in place for us to run our test and see the results.  

In [13]:
def testTotalUptime() = {
  val hosts = Map("host1" -> 10, "host2" -> 6)
  val client = new TestUptimeClient(hosts)
  val service = new UptimeService(client)
  val actual = service.getTotalUptime(hosts.keys.toList)
  val expected = hosts.values.sum
  println(actual)
  println(expected)
  assert(expected == actual)
}

// run our test
testTotalUptime()

16
16


defined function testTotalUptime